In [2]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 14.3 kB/s eta 0:00:00 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB ? eta 0:00:00MB/s eta 0:00:010:00:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 26.6 kB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 17.1 kB/s eta 0:00:0031m41.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 484.8 kB/s eta 0:00:00eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 3.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 999.8 kB/s eta 0:00:001m18.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2


In [7]:
# Define directories
train_dir = 'data/skin-disease-datasaet/train_set'
validation_dir = 'data/skin-disease-datasaet/test_set'

# Data augmentation and rescaling for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescaling for validation data
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to the desired size
    batch_size=32,
    class_mode='sparse'      # Use 'sparse' if labels are integers, otherwise use 'categorical'
)

# Load validation data
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),  # Resize images to the desired size
    batch_size=32,
    class_mode='sparse'      # Use 'sparse' if labels are integers, otherwise use 'categorical'
)


Found 924 images belonging to 8 classes.
Found 233 images belonging to 8 classes.


In [8]:
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Save the trained model
model.save('skin_disease_model.h5')


2024-06-07 22:47:41.306379: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-06-07 22:47:59.659038: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/25
28/28 [==============================] - ETA: 0s - loss: 2.4171 - accuracy: 0.1334

2024-06-07 22:49:18.509678: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


28/28 [==============================] - 81s 2s/step - loss: 2.4171 - accuracy: 0.1334 - val_loss: 2.0624 - val_accuracy: 0.1429
Epoch 2/25
28/28 [==============================] - 36s 1s/step - loss: 2.0666 - accuracy: 0.1547 - val_loss: 2.0182 - val_accuracy: 0.1696
Epoch 3/25
28/28 [==============================] - 35s 1s/step - loss: 2.0344 - accuracy: 0.1805 - val_loss: 1.9439 - val_accuracy: 0.2143
Epoch 4/25
28/28 [==============================] - 35s 1s/step - loss: 2.0300 - accuracy: 0.1749 - val_loss: 1.9916 - val_accuracy: 0.1607
Epoch 5/25
28/28 [==============================] - 36s 1s/step - loss: 1.9915 - accuracy: 0.1895 - val_loss: 1.9049 - val_accuracy: 0.2723
Epoch 6/25
28/28 [==============================] - 36s 1s/step - loss: 1.9987 - accuracy: 0.1906 - val_loss: 1.9984 - val_accuracy: 0.2098
Epoch 7/25
28/28 [==============================] - 36s 1s/step - loss: 2.0026 - accuracy: 0.2175 - val_loss: 1.9944 - val_accuracy: 0.2277
Epoch 8/25
28/28 [=============

In [14]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    predictions = model.predict(img_array)
    return predictions

img_path = 'Nail.jpg'
for i in range(4):
    predictions = predict_image(img_path, model)
    predicted_class = np.argmax(predictions)
    class_labels = list(train_generator.class_indices.keys())
    print(f"Predicted Class: {class_labels[predicted_class]}")


1/1 [==============================] - 0s 49ms/step
Predicted Class: FU-athlete-foot
1/1 [==============================] - 0s 38ms/step
Predicted Class: FU-athlete-foot
1/1 [==============================] - 0s 61ms/step
Predicted Class: FU-athlete-foot
1/1 [==============================] - 0s 57ms/step
Predicted Class: FU-athlete-foot


In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('skin_disease_model.h5')

# Assuming the class labels are already known (from train_generator)
class_labels = list(train_generator.class_indices.keys())

def predict_top_3(image_path, model):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    
    # Make predictions
    predictions = model.predict(img_array)
    
    # Get the top 3 predictions
    top_3_indices = predictions[0].argsort()[-3:][::-1]
    top_3_probs = predictions[0][top_3_indices]
    
    # Get the top 3 class labels
    top_3_labels = [class_labels[i] for i in top_3_indices]
    
    return list(zip(top_3_labels, top_3_probs))

# Example usage
image_path = 'Nail.jpg'
top_3_predictions = predict_top_3(image_path, model)

# Print the top 3 predictions
for label, prob in top_3_predictions:
    print(f"{label}: {prob:.2%}")


1/1 [==============================] - 0s 67ms/step
FU-athlete-foot: 35.12%
FU-nail-fungus: 22.61%
VI-chickenpox: 12.41%
